<h1>Part 2: This part explains how to use and implement a realtime simulation GUI for PyCX for modeling crowd behaviour. You can think of this method as a way to analyze the dynamics of the BOIDS model without having to run many simulations. This part shows how our Agent.py models these behaviours and how to create one.</h1>

<ul>The Boids Simulation Problem: To model swarming schooling behavior, so long as you follow these three basic rules:</ul>

<li>Move in the same direction as your closest neighbors.</li>
<li>Don’t stray off by yourself - stay close</li>
<li>But not too close. Avoid collisions with your neighbors.</li>

<h2>The phenomenon to be modeled and simulated:</h2>
Suppose we wish to model the crowd flocking and evacuation behaviour in a population distributed with the above three basic rules. Each individual boid must move in the same direction as its closest neighbors, meaning that the model exhibits a 'follow the leader' sort of behaviour. The second rule is to not stray off by oneself, meaning the flock of boids must move in a group, combining this with the first rule gives a flock that always faces the same direction. The third rule is to not be to close as to avoid collision with other boids. These are the main things addressed in part 1.

<h2>Conceptual model</h2>

Separation: steer to avoid crowding local flockmates

Alignment: steer towards the average heading of local flockmates

Cohesion: steer to move toward the average position of local flockmates



Let's start by importing the necessary files and creating the agent class

The following code cell defines the boundaries x and y for the old and the new agent, first initialising them.
In the Method __init__ we initialise the parameters with new values.

In [1]:
import matplotlib
import pylab as PL
import random as RD
import numpy as NP
from scipy.spatial import distance as dist
from sklearn import preprocessing

class Agent:
    
    posX = 0.0
    poxY = 0.0
    oldVelX = 0.0
    oldVelY = 0.0
    newVelX = 0.0
    newVelY = 0.0
    flock = []
    
    def __init__(self, posx, posy, oldvelx, oldvely, newvelx, newvely):
        self.posX = posx
        self.posY = posy
        self.oldVelX = oldvelx
        self.oldVelY = oldvely
        self.newVelX = newvelx
        self.newVelY = newvely

This method takes care of finding the average position of the flock so that we can use this to implement our flocking behaviour. It is neccessary for the implementation of the align rule, which causes boids that are part of the same flock to have the same general direction. For every flockmate within the alignment range, a boid will feel a force to match its heading to that of the flockmate. If there are multiple flockmates in the alignment range, the boid tries to move towards the average direction of those flockmates. The boid in the center wants to align itself with each of these flockmates.

In [2]:
def approach(self, flock):
        """Get average position of the flock"""
        distX = flock[0] / len(flock) - self.posX
        distY = flock[1] / len(flock) - self.posY    
        return [distX * 0.1, distY * 0.1]

Implementation of the Align rule:
Alignment attempts to change a boid's velocity (i.e., direction and magnitude) to match that of its nearby boids (using the same definition of "nearby" as before). Cohesion drives each boid towards its nearest neighbours, but there's a couple of notable problems we ran into :boids moving in substantially different directions tend to perform dramatic fly-bys, rather than "flocking" together: the cohesive force in the above example isn't enough to slow boids moving in opposing directions. Once the boids pass each other, they no longer "see" each other, so the turning stops.
given that a boid's visible range extends mostly in its direction of travel, the cohesive forces generated will predominantly increase its speed.

In [3]:
 def align(self, avgVel, populationSize):
        """Get average direction of flock"""
        alignX = avgVel[0] / populationSize
        alignY = avgVel[1] / populationSize

        return [alignX, alignY]

This method makes sure the boids do not collide: if they get too close, we redistribute theirdistance via proxVect.
Avoidance radiaus is the radius around the agent where the proximity is deemed too close and the agent will try to move away. Suppose the member of a flock is aiming directly for the center of the obstacle object.
1. We define P, C, r, and V as follows:

P = (1, 1)

C = (3, 3)

r = 0.5

V = (1, 1)

2. Determine if a collision might occur

s   = |C – P|

    = |(3, 3) – (1, 1)|

    = |(2, 2)|

    = (22 + 22) ½

      = 8½

      = 2.828

 

k   = (C – P) • V/ |V |

    = ((3,3) – (1, 1)) • (1, 1)/|(1, 1)|

    = (2, 2) • (1, 1)/ (12 + 12) ½

    = (2, 2) • (1, 1)/ (2) ½

    = (2, 2) • (0.707, 0.707)

    = 1.414+ 1.414

    = 2.828

 

t   = (s2 – k2) ½

                = (2.8282 – 2.8282) ½

                = 0  

 

t < r

0 < 0.5 is true, so we have a hit.  In fact, we will have a hit for any radius. This would be the definition of a collision.

In [4]:
def collisions(self, ag, avoidanceRadius, agents):
        closestRad = avoidanceRadius
        proxVect = [self.oldVelX, self.oldVelY]
        for other in agents:
            if other.posX != self.posX and other.posY != self.posY:
                distBtw = dist.euclidean([self.posX, self.posY], [other.posX, other.posY])
                if distBtw < closestRad:
                    #avoid that one
                    closestRad = distBtw
                    proxVect = [(self.posX - other.posX) * (avoidanceRadius - distBtw),
                                (self.posY - other.posY)  * (avoidanceRadius - distBtw)]
        return proxVect

Forms the flock and positions them and returns their average location and velocity to keep track of their flock movement. (flocking behaviour). 
flock centering

·        members try to stay as close as possible to the center of the flock or school

·        centering the flock locally reduces computational complexity

·        this type of tendency enables the flock to re-group if separated by the collision avoidance tendency


In [5]:
def getFlock(self, agents, flockRadius):
        """Get average location and velocity of nearby agents"""
        """Get agents nearby current agent - we only care about these ones"""
        avgLoc = [0,0]
        avgVel = [0,0]
        for other in agents:
            if dist.euclidean([self.posX, self.posY], [other.posX, other.posY]) < flockRadius:
                avgLoc[0] += other.posX
                avgLoc[1] += other.posY
                avgVel[0] += other.oldVelX
                avgVel[1] += other.oldVelY
                
        return [avgLoc, avgVel]

Potential Collision Detection and Avoidance

- simulating the cognition (thinking) of the group members, makes them more natural appearing than the pure physics-based approach because the group members sense the environment in front of themselves and plan for it

- sensing: a flock member could emit virtual “feelers” to sample the environment around itself
- model each potentially colliding object with a bounding sphere or the silhouette edges of the collision object
- when a potential collision is detected, run a “steer-to-avoid” procedure


In [6]:
def avoidObstacles(self, obstacles, avoidanceRadius, flag):
        closestRad = avoidanceRadius
        proxVect = [self.oldVelX, self.oldVelY]
        for other in obstacles:
            if other[0] != self.posX and other[1] != self.posY:
                distBtw = dist.euclidean([self.posX, self.posY], [other[0], other[1]])
                if distBtw < closestRad:
                    #avoid that one
                    closestRad = distBtw
                    proxVect = [(self.posX - other[0]) * (avoidanceRadius - distBtw),
                                (self.posY - other[1])  * (avoidanceRadius - distBtw)]
                    flag[0] = True
        return proxVect

Finish up Agent model and normalize

In [9]:
def goal(self, goalPos):
        goalVect = [0,0]
        goalVect = self.normalize(goalPos[0]-self.posX, goalPos[1]-self.posY)
        strength = 50000/dist.euclidean([self.posX, self.posY], [goalPos[0], goalPos[1]])
        goalVect = [strength * goalVect[0], strength * goalVect[1]]
        return goalVect
    
def normalize(self, x, y):
    mag = float(NP.sqrt(x*x + y*y))
    newX = x/mag
    newY = y/mag
    return [newX, newY]